<a href="https://colab.research.google.com/github/why-arong/fake-voice-detection/blob/mel-resnet/merged_mel_resnet_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare data

In [1]:
!wget 'https://drive.usercontent.google.com/download?id=1HLBDBTnrLvVdqXxMQTDJyTUf5ryBqcxD&export=download&authuser=1&confirm=t' -O open.zip

--2024-07-05 14:11:45--  https://drive.usercontent.google.com/download?id=1HLBDBTnrLvVdqXxMQTDJyTUf5ryBqcxD&export=download&authuser=1&confirm=t
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.130.132, 2404:6800:4003:c01::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.130.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3313752771 (3.1G) [application/octet-stream]
Saving to: ‘open.zip’

open.zip            100%[===================>]   3.09G  90.7MB/s    in 49s     

2024-07-05 14:12:34 (65.1 MB/s) - ‘open.zip’ saved [3313752771/3313752771]



In [2]:
!unzip -q open.zip

# Import libraries and setting

In [115]:
import os
import random
import math

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchaudio
import torchaudio.transforms as T

from torchsummary import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [116]:
SR = 32000
SEED = 1
BATCH = 32
EPOCH = 10
LR = 1e-4

In [117]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

# Define dataset and augmetation pipeline

In [118]:
def convert_to_fixed_length(audio, fixed_len):
    _, length = audio.size()

    if length < fixed_len:
        to_pad = fixed_len - length
        left_pad = torch.randint(0, to_pad + 1, (1,)).item()
        right_pad = to_pad - left_pad
        audio = torch.nn.functional.pad(audio, (left_pad, right_pad), mode='constant', value=0)
    else:
        start = torch.randint(0, length - fixed_len + 1, (1,)).item()
        audio = audio[:, start:start + fixed_len]

    return audio


class AudioDataset(Dataset):
    def __init__(self, file_paths, labels, fixed_len=64000):
        self.file_paths = file_paths
        self.labels = labels
        self.fixed_len = fixed_len
        self.length = len(file_paths)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        rand_val = random.random()

        if rand_val <= 0.25:
            wave = torch.zeros(1, self.fixed_len)
            merged_label = (0, 0)

        elif rand_val <= 0.5:
            path, label = self.file_paths[idx], self.labels[idx]
            wave, _ = torchaudio.load(path)
            wave = convert_to_fixed_length(wave, self.fixed_len)
            merged_label = (0, 1) if label else (1, 0)

        else:
            rand_idx = np.random.randint(0, self.length)
            path1, label1 = self.file_paths[idx], self.labels[idx]
            path2, label2 = self.file_paths[rand_idx], self.labels[rand_idx]

            wave1, _ = torchaudio.load(path1)
            wave2, _ = torchaudio.load(path2)

            wave1 = convert_to_fixed_length(wave1, self.fixed_len)
            wave2 = convert_to_fixed_length(wave2, self.fixed_len)

            wave = (wave1 + wave2) / 2
            merged_label = (int(label1 == 0 or label2 == 0), int(label1 == 1 or label2 == 1))

        label = torch.tensor(merged_label).float()
        return wave, label

In [119]:
class AudioPipeline(nn.Module):
    def __init__(self, on_train=True, noise_level=0.1, shape=(128, 128), fixed_len=64000):
        super().__init__()
        self.on_train = on_train
        self.noise_level = noise_level

        self.spec = T.MelSpectrogram(
            sample_rate=SR,
            n_fft=2048,
            win_length=1024,
            hop_length=fixed_len // shape[1] + 1,
            n_mels=shape[0]
        )
        self.to_db = T.AmplitudeToDB()
        self.spec_aug = nn.Sequential(
            T.FrequencyMasking(32),
            T.TimeMasking(12),
            T.TimeMasking(12),
        )

    def forward(self, wave):
        with torch.no_grad():
            if self.on_train:
                noise = torch.randn_like(wave) * self.noise_level
                wave = wave + noise

            spec = self.spec(wave)
            spec = self.to_db(spec)

            if self.on_train:
                spec = self.spec_aug(spec)

            spec = self.normalize(spec)
        return spec

    def normalize(self, spec, epsilon=1e-6):
        mean = spec.mean(dim=[2, 3], keepdim=True)
        std = spec.std(dim=[2, 3], keepdim=True)
        return (spec - mean) / (std + epsilon)

    def train(self):
        self.on_train = True

    def eval(self):
        self.on_train = False

In [120]:
train = pd.read_csv("train.csv")
train["label"] = train["label"].apply(lambda x: 1 if x == "real" else 0)
train.head()

,id,path,label
0,RUNQPNJF,./train/RUNQPNJF.ogg,1
1,JFAWUOGJ,./train/JFAWUOGJ.ogg,0
2,RDKEKEVX,./train/RDKEKEVX.ogg,1
3,QYHJDOFK,./train/QYHJDOFK.ogg,1
4,RSPQNHAO,./train/RSPQNHAO.ogg,1


In [121]:
train_paths, valid_paths, train_labels, valid_labels = train_test_split(
    train["path"].values, train["label"].values, test_size=0.2, stratify=train["label"]
)

train_dataset = AudioDataset(train_paths, train_labels)
valid_dataset = AudioDataset(valid_paths, valid_labels)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

audio_pipeline = AudioPipeline().to(device)

# Define model

In [122]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)

        return x


class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = ConvBlock(in_channels, out_channels, stride=stride)
        self.conv2 = ConvBlock(out_channels, out_channels)

        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Identity()

        self.relu = nn.ReLU()

    def forward(self, x):
        shortcut = self.shortcut(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x + shortcut
        x = self.relu(x)
        return x

In [123]:
class AudioClassifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.conv1 = ConvBlock(1, 16)
        self.res1 = ResBlock(16, 16)
        self.res2 = ResBlock(16, 32, stride=2)
        self.res3 = ResBlock(32, 32)
        self.res4 = ResBlock(32, 64, stride=2)
        self.res5 = ResBlock(64, 64)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(64, 256)
        self.fc2 = nn.Linear(256, n_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)
        x = self.res4(x)
        x = self.res5(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [124]:
model = AudioClassifier(2).to(device)

summary(model, input_size=(1, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 128, 128]             160
       BatchNorm2d-2         [-1, 16, 128, 128]              32
              ReLU-3         [-1, 16, 128, 128]               0
         ConvBlock-4         [-1, 16, 128, 128]               0
          Identity-5         [-1, 16, 128, 128]               0
            Conv2d-6         [-1, 16, 128, 128]           2,320
       BatchNorm2d-7         [-1, 16, 128, 128]              32
              ReLU-8         [-1, 16, 128, 128]               0
         ConvBlock-9         [-1, 16, 128, 128]               0
           Conv2d-10         [-1, 16, 128, 128]           2,320
      BatchNorm2d-11         [-1, 16, 128, 128]              32
             ReLU-12         [-1, 16, 128, 128]               0
        ConvBlock-13         [-1, 16, 128, 128]               0
             ReLU-14         [-1, 16, 1

# Train

In [125]:
def run_epoch(model, dataloader, pipeline, optimizer, criterion, train=True):
    total_loss = 0
    count = 0
    correct = 0

    if train:
        model.train()
        pipeline.train()
    else:
        model.eval()
        pipeline.eval()

    for waves, labels in tqdm(dataloader, desc="Training" if train else "Evaluating"):
        waves = waves.to(device)
        specs = audio_pipeline(waves)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(specs)
        loss = criterion(outputs, labels)

        if train:
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        count += len(labels) * 2
        correct += ((outputs > 0).float() == labels).sum().item()

    return total_loss / count, correct / count

In [126]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

best_accuracy = 0
best_model = None

for epoch in range(EPOCH):
    print(f"Epoch {epoch}")

    train_loss, train_accuracy = run_epoch(model, train_dataloader, audio_pipeline, optimizer, criterion, train=True)

    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    print(f"Train Loss: {train_loss:.8f}")
    print(f"Train Accuracy: {train_accuracy:.8f}")

    valid_loss, valid_accuracy = run_epoch(model, valid_dataloader, audio_pipeline, optimizer, criterion, train=False)
    valid_losses.append(valid_loss)
    valid_accuracies.append(valid_accuracy)

    print(f"Test Loss: {valid_loss:.8f}")
    print(f"Test Accuracy: {valid_accuracy:.8f}")

    if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_model = model

Epoch 0


Training: 100%|██████████| 1386/1386 [07:23<00:00,  3.13it/s]


Train Loss: 0.00628956
Train Accuracy: 0.80489290


Evaluating: 100%|██████████| 347/347 [01:26<00:00,  3.99it/s]


Test Loss: 0.01349071
Test Accuracy: 0.61918290
Epoch 1


...

Train Loss: 0.00349705
Train Accuracy: 0.90178129


Evaluating: 100%|██████████| 347/347 [01:28<00:00,  3.91it/s]


Test Loss: 0.03189618
Test Accuracy: 0.38271104
Epoch 5


Training:  42%|████▏     | 578/1386 [03:06<04:20,  3.10it/s]


KeyboardInterrupt: 

In [ ]:
epochs_range = range(EPOCH)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_losses, label='Train Loss')
plt.plot(epochs_range, valid_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_accuracies, label='Train Accuracy')
plt.plot(epochs_range, valid_accuracies, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()

In [17]:
torch.save(best_model.state_dict(), "merged_mel_resnet_v3.pth")

# Inference

In [33]:
class TestAudioDataset(Dataset):
    def __init__(self, file_paths, fixed_len=64000):
        self.file_paths = file_paths
        self.fixed_len = fixed_len
        self.length = len(file_paths)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        wave, _ = torchaudio.load(self.file_paths[index])

        start_idx = int(1.5 * SR)
        end_idx = int(3.5 * SR)
        wave = wave[:, start_idx:end_idx]
        return wave

In [34]:
test = pd.read_csv("test.csv")
test_dataset = TestAudioDataset(test["path"].values)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
best_model.eval()
audio_pipeline.eval()
preds = []

for waves in tqdm(test_dataloader):
    specs = audio_pipeline(waves.to(device))
    outputs = best_model(data)
    pred = F.sigmoid(outputs).detach().cpu().numpy()
    preds += pred.tolist()

In [ ]:
submit = pd.read_csv("sample_submission.csv")
submit.iloc[:, 1:] = preds
submit.to_csv("merged_mel_resnet_v2_max.csv", index=False)

In [ ]:
submit.head()

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.title("Distribution of Fake")
plt.hist(submit["fake"], bins=20)

plt.subplot(1, 2, 2)
plt.title("Distribution of Real")
plt.hist(submit["real"], bins=20)
plt.show()